OS detection Justin Wang

Having generated the nprints, and the metadata which is written in file_name,label, we now prepare for training

In [1]:
import os
import pandas as pd
import numpy as np
split_dir = os.path.expanduser("/workspaces/course-project-Doawgh/final/split_output")
nprint_dir = os.path.expanduser("/workspaces/course-project-Doawgh/final/nprints")
metadata_file = os.path.join(split_dir, "metadata.csv")

In [ ]:
from sklearn.decomposition import TruncatedSVD
# load metadata
metadata = pd.read_csv(metadata_file)
metadata.set_index("File", inplace=True)

X = []
y = []

nprint_files = os.listdir(nprint_dir)
count = 0
for filename in nprint_files:

    # only process .nprint files
    if not filename.endswith(".nprint"):
        continue
    
    nprint_path = os.path.join(nprint_dir, filename)

    # load nPrint CSV
    nprint_data = pd.read_csv(nprint_path)

    # keep only numeric columns and flatten
    flat = nprint_data.select_dtypes(include=[np.number]).values.flatten()
    X.append(flat)

    # map filename to label
    key = filename.replace(".nprint", ".pcap")
    if key in metadata.index:
        label = metadata.loc[key, "Label"]
        y.append(label)
    else:
        raise ValueError(f"Label not found for {key}")
    
    # added for the sake of time so that training doesn't take hours/days
    # if you want to increase quality of prediction then raise this number
    count += 1
    if count >= 200:
        break

# convert lists to df for AutoGluon
X_df = pd.DataFrame(X)
X_df['Label'] = y

X_features = X_df.drop(columns='Label')
y_labels = X_df['Label']
X_features_filled = X_features.fillna(X_features.mean())
# reduces the number of features, meaning the training takes less time to run. Up to around 96k for n_components
svd = TruncatedSVD(n_components=2000, random_state=42)
X_reduced = svd.fit_transform(X_features_filled)


X_reduced_df = pd.DataFrame(X_reduced)
X_reduced_df['Label'] = y_labels

In [ ]:
from sklearn.model_selection import train_test_split
from autogluon.tabular import TabularPredictor
data = X_reduced_df.copy()
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42, stratify=data['Label'])
label_column = 'Label'
predictor = TabularPredictor(label=label_column, eval_metric='balanced_accuracy').fit(
    train_data,
    # goes up to higher qualities, like best quality
    presets='good_quality',
    # increase training time if you want higher accuracy
    # unfortunately my computer is garbage so this took me really long 
    time_limit=600            
)
performance = predictor.evaluate(test_data)
print(performance)

No path specified. Models will be saved in: "AutogluonModels/ag-20251212_123846"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.4.0
Python Version:     3.12.3
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Thu Jun  5 18:30:46 UTC 2025
CPU Count:          16
Memory Avail:       3.58 GB / 7.35 GB (48.7%)
Disk Space Avail:   933.16 GB / 1006.85 GB (92.7%)
Presets specified: ['good_quality']
Using hyperparameters preset: hyperparameters='light'
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1


Note: `save_bag_folds=False`! This will greatly reduce peak disk usage during fit (by ~8x), but runs the risk of an out-of-memory error during model refit if memory is small relative to the data size.
	You can avoid this risk by setting `save_bag_folds=True`.
DyStack is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
	This is used to identify the optimal `num_stack_levels` value. Copies of AutoGluon will be fit on subsets of the data. Then holdout validation data is used to detect stacked overfitting.
	Running DyStack for up to 150s of the 600s of remaining time (25%).
		Context path: "/home/doawgh/nprint/AutogluonModels/ag-20251212_123846/ds_sub_fit/sub_fit_ho"
(_dystack pid=4986) Running DyStack sub-fit ...
(_dystack pid=4986) Beginning AutoGluon training ... Time limit = 150s
(_dystack pid=4986) AutoGluon will save models to "/home/doawgh/nprint/AutogluonModels

{'balanced_accuracy': np.float64(0.43589743589743607), 'accuracy': 0.45, 'mcc': 0.402978130659247}
